## Pythton面向对象编程——和C++做对比

### 类和实例对象的语法
对于C++来说，类中声明的变量中，没有static关键字的（非静态成员变量）就是实例变量，有static关键字的（类的静态成员变量）是类本身的成员变量。函数也类似。

对于Python，类中声明的变量是类变量，在__init__()方法中声明的变量才是实例变量。

用@classmethod装饰的函数是类函数，类函数只能使用类变量；

用@staticmethod装饰的函数是静态函数，静态函数既不能使用类变量，也不能使用实例变量。 经常用于封装工具函数。

用@property装饰的函数是属性函数，可以调用实例变量和类变量，特点是调用时不用添加括号。

Python中的self（即this指针）是显式传递的。

In [1]:
class Person:
    nationality = 'China' 
    __amount=0 #这两个都是类属性，所有实例共享。
    def __init__(self, name, age, sex="M", *args,**kwargs): #构造函数，但不能和c++一样重载，只能有一个，因此这里使用了动态参数和默认参数。
        self.name = name
        self.age = age
        self.sex = sex
        if args:  # 位置参数
            self.a = args[0]
        if 'b' in kwargs:  # 关键字参数
            self.b = kwargs['b']
        self.__class__.__increase()#self.__class__是通过一个实例直接锁定到类的方法。c++没有类似的方法。
    
            
    #@classsmethod装饰器装饰的函数是类方法。        
    @classmethod
    def passport(cls): #类方法，第一个参数为cls，(这里也可以用self，self也表示类本身)代表类本身，也可以通过类名调用。
        print(f"My passport is authorized by {cls.nationality}.")
    @classmethod
    def __increase(cls):
        cls.__amount+=1
        print(f"Now there are {cls.__amount} people in the class.")
        
    @staticmethod
    def add(x, y): #静态方法不能使用类属性，也不能使用实例属性，割断了这个函数和类与实例的关系。
        return x + y
    
    @property 
    def graduate_age(self):
        return self.age+3
    
    
    #self作为第一个函数传递。
    def introduce(self):
        print(f"Hello, my name is {self.name} and I am {self.age} years old.")
        
Person1=Person("Alice", 25, "F", 10, b=20)
Person2=Person("Bob", 30, "M", 30, b=40)
Person3=Person("Charlie", 35, "M", 50, b=60)
Person4=Person("Diana", 40, "F", 70, b=80)
print(Person1.graduate_age)

Now there are 1 people in the class.
Now there are 2 people in the class.
Now there are 3 people in the class.
Now there are 4 people in the class.
28


Python是一个动态化的语言，不支持函数重载，一般有两种方法解决函数重载问题：
1. 动态参数，即参数个数可变，使用*args和**kwargs
2. 使用默认参数
3. 类方法（@classmethod）作为替代构造函数​
4. 使用 __new__ 方法
后两种后面再说

### 用实例对象访问类方法
C++和Python在使用时都不建议用实例名访问类属性，而是建议用类名访问。

但两者用实例修改类变量的操作效果是不一样的，C++中会直接修改类的静态变量，而Python是创建一个类变量的同名变量并作修改。

In [2]:
p1=Person("John",20)
p2=Person("Alice",25)
print(Person.nationality)
print(p1.nationality)
p1.nationality="JP"
print(Person.nationality)
print(p1.nationality)

Now there are 5 people in the class.
Now there are 6 people in the class.
China
China
China
JP


### 类（的实例对象）之间的关系：
1.依赖关系：如主人和狗的关系。

一个类实例化时必须指定一个其他类的实例对象作为参数。

2.关联关系：如男女朋友的关系。

两者的关系并不是实例化所必需的，而是在运行时建立的关系。
实现方法：将关联关系本身新建为一个类。

3.组合关系：人身体内各个器官的关系。

两者之间合作（可能有关联关系），并同时依赖于另一个类，当另一个类的实例对象被删除时，两者也随之删除。

4.聚合关系：如电脑各个部件的关系。

各类同时聚合为一个功能体，类之间有关联关系，但并不依赖于聚合而成的类，单独也能存在。

5.继承关系：如猫科动物和波斯猫的关系。

当在子类中调用父类的构造函数时，C++需要显示指定类名，如Base::Base(),没有Super()这样的写法。




In [ ]:
# 继承关系示例
class Animal:
    a_type="哺乳动物"
    def __init__(self, name):
        self.name = name
    def speak(self):
        print(f"{self.name} makes a sound.")

class Dog(Animal):
    def __init__(self, name, breed='金毛'):
        #super()用于找到父类的方法
        super().__init__(name)
        self.breed = breed
        print("Here is a dog.")
    def speak(self):
        print(f"{self.name} barks.")

class Cat(Animal):
    def __init__(self, name, color='white'):
        super().__init__(name)
        self.color = color
        print("Here is a cat.")
    def speak(self):
        print(f"{self.name} meows.")

# 对于多重继承，如果只是一个类继承两个类，可以认为会按照从左到右继承，但底层用的是C3算法，比较复杂，可以通过__mro__查看继承顺序
# 多重继承示例
class Something(Dog,Cat):
    def __init__(self, name, breed, color):
        super().__init__(name, breed)
        self.color = color
#查看继承顺序
print(Something.__mro__)
Something("猫狗兽","ssr","金色").speak()

(<class '__main__.Something'>, <class '__main__.Dog'>, <class '__main__.Cat'>, <class '__main__.Animal'>, <class 'object'>)
Here is a cat.
Here is a dog.
猫狗兽 barks.


### 访问权限的控制
另外，Python中没有public、private、protected关键字，而是使用下划线来表示不同的访问权限。

- 单下划线开头的变量或方法是受保护的，只能在类内部和子类中访问。
- 双下划线开头的变量或方法是私有的，只能在类内部访问。
但这两种也并不是硬性的，事实上按特定格式也可以访问。

In [4]:
class Dog():
    def __init__(self, name, breed='金毛'):
        self.name = name
        self.breed = breed
        print("Here is a dog.")
    def __speak(self):
        print(f"{self.name} barks.")
        
myDog = Dog('Rex')
print(myDog.name)
print(myDog.breed)
#myDog.__speak()不可以直接访问私有方法
myDog._Dog__speak()#可以通过这种方式访问私有方法(所以双下划线就像是改了个名字)

myDog.__color='white'#可以直接添加属性
print(myDog.__color)#但是可以直接访问，说明私有属性是不能在类外定义的，这样没有启动改名的机制
#print(myDog._Dog__color) 这个就不能访问

Here is a dog.
Rex
金毛
Rex barks.
white


### Python中多态的实现
在C++中，多态是通过虚函数和基类指针来表现的。

在Python中，多态的实现是通过各种类型共同的容器来实现的。

In [5]:
class Dog:
    def speak(self):
        return "Woof!"

class Cat:
    def speak(self):
        return "Meow!"
    
class Otto:
    def speak(self):
        return "WOW!"
    
list=[Dog(),Cat(),Otto()]
for animal in list:
    print(animal.speak())

Woof!
Meow!
WOW!


### 反射
反射是指程序在运行时动态地访问、检测和修改自身状态或行为的能力。具体来说，它允许coder通过字符串形式的名称来操作对象、类、模块等。

Python的反射能力源于其"一切都是对象"的设计，模块、类、函数、实例​​都是对象，可以通过字符串名称访问。对象的属性和方法存储在 __dict__ 字典中，反射操作本质是对 __dict__ 的读写。

常见的反射方法包括：
- `hasattr(obj, name)`: 检查对象是否有指定的属性。
- `getattr(obj, name, default=None)`: 获取对象的属性值，如果属性不存在则返回默认值。
- `setattr(obj, name, value)`: 设置对象的属性值。
- `delattr(obj, name)`: 删除对象的属性。
- `​obj.__class__`: 返回实例对象的类名。
- `importlib.import_module()`: 动态导入模块

In [ ]:
#反射对类和实例对象操作的示例
class Person:
    def __init__(self, name):
        self.name = name
    def say_hello(self):
        print(f"Hello, {self.name}!")

# 创建一个Person对象
person = Person("Alice")

# 使用反射获取对象的属性和方法
# 判断对象是否有指定的属性或方法 hasattr(obj, name)
print("Has attribute 'name':", hasattr(person, "name"))  # 输出: Has attribute 'name': True

# 获取对象的属性值或调用方法 getattr(obj, name[, default])
print("Name:", getattr(person, "name"))  # 输出: Name: Alice
func=getattr(person, "say_hello")  # 获取方法对象
func()  # 调用方法

# 修改或添加对象和属性和方法 setattr(obj, name, value)
setattr(person, "name", "Bob")  # 修改属性值
print("Name after modification:", getattr(person, "name"))  # 输出: Name after modification: Bob
setattr(person, "age", 15)  # 添加一个新属性
print("Has attribute 'age':", hasattr(person, "age"))  # 输出: Has attribute 'age': True
print("Age:", getattr(person, "age"))  # 输出: Age: 15
setattr(person, "say_goodbye", lambda: print("Goodbye!"))  # 添加一个新方法
func=getattr(person, "say_goodbye")  # 获取方法对象
func()

# 删除对象的属性和方法 delattr(obj, name)
delattr(person, "age")  # 删除属性
print("Has attribute 'age':", hasattr(person, "age"))  # 输出: Has attribute 'age': False

Has attribute 'name': True
Name: Alice
Hello, Alice!
Name after modification: Bob
Has attribute 'age': True
Age: 15
Goodbye!
Has attribute 'age': False


通过__name__可以判断当前模块是否作为主模块执行

In [17]:
# 反射对模块操作示例

print(__name__)

import numpy as np


# 判断模块中是否有Person类
if hasattr(np, "ndarray"):
    print("存在")
    
function_name = "sqrt"  # 目标函数名（字符串）
if hasattr(np, function_name):  # 检查函数是否存在
    np_func = getattr(np, function_name)  # 获取函数对象
    result = np_func(4)  # 执行函数（计算平方根）
    print(f"np.{function_name}(4) = {result}")  # 输出: np.sqrt(4) = 2.0
else:
    print(f"NumPy has no function: {function_name}")

__main__
存在
np.sqrt(4) = 2.0


### 类的双下划线方法
双下划线方法（Dunder Methods / Magic Methods）​​ 是类中用于定义特殊行为的内置方法，通常以 __ 开头和结尾（如 __init__）。它们允许开发者自定义类的行为，使其支持Python的核心语法（如运算符、迭代、内置函数等）。

以下是一些常见的双下划线方法：

对象的初始化和表示：
1. `__init__(self, ...)`: 构造函数，用于初始化对象的属性。对应对象的构建过程。
2. `__str__(self)`: 返回对象的字符串表示，通常用于打印对象时调用。对应print()函数。
3. `__repr__(self)`: 返回对象的“官方”字符串表示，通常用于调试和交互环境中。对应repr()函数。

运算符的重载：

4. `__add__(self, other)`: 定义对象的加法操作。对应 + 运算符。
5. `__sub__(self, other)`: 定义对象的减法操作。对应 - 运算符。
6. `__mul__(self, other)`: 定义对象的乘法操作。对应 * 运算符。
7. `__truediv__(self, other)`: 定义对象的除法操作。对应 / 运算符。
8. `__eq__(self, other)`: 定义对象的相等性比较。对应 == 运算符。
9. `__ne__(self, other)`: 定义对象的不等性比较。对应 != 运算符。
10. `__lt__(self, other)`: 定义对象的小于比较。对应 < 运算符。
11. `__le__(self, other)`: 定义对象的小于等于比较。对应 <= 运算符。
12. `__gt__(self, other)`: 定义对象的大于比较。对应 > 运算符。
    
容器类型的行为：

13. `__len__(self)`: 返回容器中元素的数量。对应 len() 函数。
14. `__getitem__(self, key)`: 定义对象的索引操作，用于访问容器中的元素。对应索引操作。
15. `__setitem__(self, key, value)`: 定义对象的索引赋值操作，用于修改容器中的元素。对应索引赋值操作。
16. `__delitem__(self, key)`: 定义对象的索引删除操作，用于删除容器中的元素。对应 del 语句。

迭代器和生成器：

17.  `__iter__(self)`: 返回一个迭代器对象，用于遍历容器中的元素。对应 iter() 函数。
18.  `__next__(self)`: 定义迭代器对象的迭代行为，用于返回下一个元素。对应 next() 函数。

### __new__方法和单例模式
事实上一个类的实例化过程可以分为三个步骤:
1. 调用__new__方法,创建一个新的对象。
2. 调用__init__方法,初始化这个对象。
3. 返回这个对象。

所以__new__方法是在__init__方法之前调用的静态方法，python底层标记为静态，显式添加@staticmethod反而会破坏其工作。

__new__方法的第一个参数是cls,表示要创建的类,第二个参数是*args和**kwargs,这些参数与__init__方法共享。

__new__方法的返回值是一个实例对象,这个对象会被传递给__init__方法。


In [19]:
class Student:
    def __init__(self, name):
        print('__init__')
        self.name = name
    def __new__(cls, *args, **kwargs):
        print('__new__')
        return super().__new__(cls)
    
stu = Student('zs')
print(stu.name)

__new__
__init__
zs


In [20]:
# 单例模式的实现
class Singleton:
    _instance = None  # 类变量，保存唯一实例

    def __new__(cls, *args, **kwargs):
        # 如果实例不存在，则创建；否则直接返回已有实例
        if cls._instance is None:
            cls._instance = super().__new__(cls)  # 调用父类创建实例
        return cls._instance  # 始终返回同一实例

    def __init__(self, value):
        # 每次实例化都会调用 __init__，因此需要避免重复初始化
        if not hasattr(self, '_initialized'):  # 检查是否已初始化
            self.value = value
            self._initialized = True  # 标记为已初始化

# 测试
s1 = Singleton("First")
s2 = Singleton("Second")

print(s1 is s2)        # 输出: True（是同一个实例）
print(s1.value)        # 输出: "First"（s2的初始化被跳过）
print(s2.value)        # 输出: "First"（值未被覆盖）

True
First
First


### __call__方法和可调用对象

_call__方法​​的作用是让一个类的实例能够像函数一样被调用（即实现“可调用对象”）。

当在实例后加括号时（如instance()），Python会自动调用该实例的__call__方法。

In [ ]:
# 将对象转化成函数的形式
class Adder:
    def __init__(self, base):
        self.base = base  # 维护内部状态

    def __call__(self, x):
        return self.base + x  # 实例调用时执行的逻辑

add_5 = Adder(5)      # 创建实例，base=5
print(add_5(3))       # 输出: 8 （等价于 add_5.__call__(3)）
print(add_5(10))      # 输出: 15

8
15


### 通过type动态创建类
所有的类（包括内置类和用户自定义类）本质上都是 ​​type 类的实例​​。这种设计源于Python的元类（Metaclass）机制，type 是默认的元类，负责创建和定义类。

In [23]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    def introduce(self):
        print(f"Hello, my name is {self.name} and I am {self.age} years old.")
        
print(type(Person))

# 用type创建一个类,第一个参数是类名,第二个参数是父类,第三个参数是类的属性和方法,返回一个类对象。
Student = type(
    'Student',                   # 类名
    (Person,),                   # 父类元组（注意逗号表示单元素元组）
    {
        'school': 'MIT',         # 类属性
        'study': lambda self, subject: print(f"{self.name} is studying {subject}")  # 方法
    }
)

# 测试动态创建的类
s = Student("Alice", 20)
s.introduce()          # 继承自 Person 的方法
print(s.school)        # 输出: MIT
s.study("Math")       # 输出: Alice is studying Math

<class 'type'>
Hello, my name is Alice and I am 20 years old.
MIT
Alice is studying Math
